Setup
===


Zadanie biznesowe: 

*Wygląda na to, że nasze firmy kurierskie czasami nie radzą sobie z dostawami. Gdybyśmy wiedzieli, ile taka dostawa dla danego zamówienia potrwa – moglibyśmy przekazywać tą informację klientom.*

Zadanie modelowania:

Zadanie regregresji - przewidzieć czas dostawy zamówienia. Kryteria oceny modelu mogą być takie jak zazwyczaj dla regresji np. MSE, lub absolute mean error. Dodatkowo można zamiast jednego czasu dostawy przewidywać okno dostawy o założonej szerokości np. 2h i zastosować własną, niestandardową funkcję celu, która traktuje wszystkie dostawy, które wpadły w okienko jako przewidziane poprawnie.

In [1]:
import pandas as pd



In [ ]:
data_dir ="drive/MyDrive/Data/IUM/"

deliveries = pd.read_json(data_dir + "deliveries.jsonl", lines=True)
sessions = pd.read_json(data_dir + "sessions.jsonl", lines=True)
users = pd.read_json(data_dir + "users.jsonl", lines=True)
products = pd.read_json(data_dir + "products.jsonl", lines=True)

data = {"products": products, "sessions": sessions, "users": users, "deliveries":deliveries}

In [ ]:
deliveries.head()

Eksploracyjna analiza danych 
======

In [ ]:
print(deliveries.shape, sessions.shape, users.shape, products.shape)
print(deliveries.columns, "\n", sessions.columns,"\n", users.columns,"\n", products.columns)

Ilościowo danych wydaje się być dostatecznie dużo, w najbardziej interesującej nas tabeli (deliveries) jest niemal 7 tysięcy wierszy, co powinno być dość zarówno do wytrenowania, walidacji jak i testowania modelu. 


In [ ]:
deliveries.head()

In [ ]:
sessions.head()

In [ ]:
users.head()

In [ ]:
products.head()

Omówienie kolumn
----

**Dane które na pewno będą istotne:**
- adres użytkownika (może być konieczne dodanie do niego kolumny z kodem pocztowym - powinno dać to lepsze pojęcie o położeniu punktów dostaw w tym samym rejonie, dalszym przetworzeniem może być też odległość absolutna pomiędzy magazynem/miejscem wysyłki a celem)
- dane dt. czasu złożenia zamówienia (prawdopodobnie trzeba będzie je przekszałcić w format bardziej nadający się do obróbki)
- dane dt. czasu dostawy (atrybut którego wartość będziemy próbowali przewidzieć wprost tj. datę lub pośrednio tj. czas między złożeniem zamówienia, a dostawą)
- dane dt. firmy kurierskiej (firma prawdopodobnie będzie mocno wpływać na czas dostawy)

**Dane które by się przydały:**
- dane dt. momentu i miejsca wysyłki produktów (czy zawsze wysyłane są z tego samego miejsca, czy istnieją jakieś opóźnienia)
- dane dt. punktów pośrednich na drodze przesyłki (o ile istnieją) np. punktów spedycyjnych, magazynów, to potencjalnie mogłoby pozwolić na ulepszanie predykcji w trakcie podróży przesyłki i im bliżej celu, tym dokładniejsze dane można by było zaprezentować użytkownikowi
- 

**Pozostałe dane:**

- session_id - raczej nieistotne 
-	timestamp z tabeli sessions - też raczej nieistotny, podobne informacje niesie timestamp zakupu produktu z tabeli deliveries, gdyż nie obchodzi nas jakie produkty przeglądał użytkownik, a tylko akcje kupna
- user_id - samo w sobie raczej nie niesie wartości, choć może być przydatne do łączenia z innymi tablicami (np. deliveries)
- product_id 	- przydatne do łączenia
- event_type 	- raczej nieistotne
- offered_discount 	- raczej nieistotne
- purchase_id - istotne do łączenia
- name 	- samo w sobie nieważne, można by z niego wyciągnąć takie dane jak np. płeć, jednak nie będzie to miało wpływu na czas dostawy
- product_name - raczej nie istotne
- category_path - może być o tyle ważne, że pewne produkty mogłyby być gabarytami, wtedy ich czas dostawy mógłby się znacznie różnic przez konieczność specjalnego ich traktowania
- price - cena sama w sobie raczej nie niesie wartości


**Ogólny komentarz**

Tabela deliveries - ważna, niesie istotne dane, choć może być konieczność poddania ich odpowiedniemu przetworzeniu przed utworzeniem modelu. Zawarte w niej daty i godziny są prawdopodobnie bardzo ważne w budowie modelu, ale będą wymagały przetworzenia - np. wyciągnięcia oddzielnie miesiąca, dnia, godziny, może też dnia tygodnia. 

Tabela sessions - raczej nieważna, choć spodziewam się że może nieść nieco istotnych danych nt. obciążenia systemu w poszczególnych okresach, co prawdopodobnie wpływa na czas dostaw. 
:owe użytkowników. 

Tabela products - przy wstępnej analizie nie wydaje się istotna, ale może da się na jej podstawie utworzyć nowe atrybuty bardziej przydatne dla problemu. Być może warto np. dodać do kolumny deliveries kolumnę zliczającą ile przedmiotów jest w jednym zamówieniu - wielkość zamówienia może wpływać na możliwości firm kurierskich co do czasu dostawy.


In [ ]:
for key in data.keys():
  print(key, ":\n", data[key].isnull().sum(), "\n")

Brakujące dane w tabeli deliveries są raczej martwiące, dobrze by było jakby dało się je uzupełnić, jednak jeśli nie będzie to możliwe to pozostałe dane powinny być wystarczające do wytrenowania modelu (max 662 wiersze są wybrakowane, to poniżej 10% danych)

Cieszy brak pustych wartości w tabelach dotyczących użytkowników - nie powinno być problemu z nieznanymi adresami dostaw.

Braki w tabeli sessions nie zaskakują - wynikają prawdopodobnie ze sposobu jaki są gromadzone dane, np. puste wartości w kolumnie purchase_id wynikają najpewniej z tego że dany wiersz odpowiada wartości event_type innej niż BUY_PRODUCT

In [ ]:
categories = set()
for value in products["category_path"]:
  categories.update(value.split(";"))

categories

Po przyjrzeniu się kategoriom można zauważyć, że gabaryty sprzedawanych przedmiotów nie będą się bardzo różnić - choć transport monitorów LCD i Słuchawek jest różny, to raczej nie dość różny by wpłynąć na czas dostawy. Mimo to można dodać kolumnę dzielącą przedmioty na większe np. Drukarki i skaner, Komputery, Monitory i Monitory LCD jako urządzenia "duże", a telefony komorkowe, zestawy słuchawkowe i gry na konsole jako "małe". 

Feature engineering
-----

Dane czasowe - zamienić na czas absolutny, policzyć kolumnę czasu dostawy, pokazać dane na grafie i sprawdzić jakie są w nich trendy.

Adres - kod pocztowy, odległość absolutna

In [ ]:
deliveries.head()


In [ ]:
deliveries["purchase_timestamp"] = pd.to_datetime ( deliveries["purchase_timestamp"], format='%Y-%m-%dT%H:%M', errors="coerce")
deliveries['delivery_timestamp'] = pd.to_datetime ( deliveries["delivery_timestamp"], format='%Y-%m-%dT%H:%M:%S', errors="coerce")
deliveries.head()

In [ ]:
deliveries["time_difference"] = deliveries["delivery_timestamp"] - deliveries["purchase_timestamp"]
deliveries.head()

In [ ]:
len(deliveries[deliveries["purchase_timestamp"] > deliveries["delivery_timestamp"]])

Ujemne wartości w nowoutworzonej kolumnie wskazują na to, że dane są zanieczyszczone - produkt nie mógł być dostarczony przed zakupem. Takich zanieczyszczeń jest stosunkowo wiele - 2 tysiące wierszy, które przez to nie będą mogły zostać wykorzystane w modelowaniu. To zbyt wiele danych, żebyśmy mogli je po prostu pominąć, jak w przypadku brakujących czasów dostawy. 

In [ ]:
deliveries[deliveries["purchase_timestamp"] > deliveries["delivery_timestamp"]]

In [ ]:
 deliveries["delivery_company"].unique()


TO-DO
=====
- feature engineering
- outliers detection 
- categorical data encoding